In [2]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [3]:
#from google.colab import files
#src = list(files.upload().values())[0]
#open('clr_callback.py','wb').write(src)
#import clr_callback
#!pip install https://drive.google.com/open?id=1CBbGtUvTYwq97sxtpNdXJKDHs0GMGlYC
 !git clone https://github.com/bckenstler/CLR 

Cloning into 'CLR'...
remote: Enumerating objects: 244, done.
remote: Total 244 (delta 0), reused 0 (delta 0), pack-reused 244
Receiving objects: 100% (244/244), 1.37 MiB | 26.44 MiB/s, done.
Resolving deltas: 100% (86/86), done.


In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import SGD
import numpy as np
from CLR.clr_callback import CyclicLR


In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 40
num_filter = 12
compression = 0.5
dropout_rate = 0.2

In [7]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]
print(img_width)

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 49s 0us/step
32


In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 26
dropout_rate = 0.2
l = 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (5,5), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Fourth_Block = add_denseblock(Third_Transition, num_filter, dropout_rate)
Fourth_Transition = add_transition(Fourth_Block, num_filter, dropout_rate)

Fifth_Block = add_denseblock(Fourth_Transition, num_filter, dropout_rate)
Fifth_Transition = add_transition(Fifth_Block, num_filter, dropout_rate)

#Sixth_Block = add_denseblock(Fifth_Transition, num_filter, dropout_rate)
#BatchNormalization()(Sixth_Block)

Last_Block = add_denseblock(Fifth_Block,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [12]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 26)   1950        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 26)   104         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 26)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [0]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
datagen.fit(x_train)

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(momentum=0.9, nesterov=True),
              metrics=['accuracy'])

In [0]:
def step_decay_schedule(initial_lr=1e-3, decay_factor=0.75, step_size=10):
    '''
    Wrapper function to create a LearningRateScheduler with step decay schedule.
    '''
    def schedule(epoch):
        return initial_lr * (decay_factor ** np.floor(epoch/step_size))
    
    return LearningRateScheduler(schedule)

In [0]:


#lr_sched = step_decay_schedule(initial_lr=1e-2, decay_factor=0.75, step_size=7)
lr_sched = CyclicLR(mode='triangular')
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs, validation_data=(x_test, y_test), callbacks=[lr_sched])
#model.fit(x_train, y_trauin,
#                    batch_size=batch_size,
 #                   epochs=epnoochs,
  #                  verbose=1,
   #                 validation_data=(x_test, y_test))

Epoch 1/50
390/390 [==============================] - 230s 589ms/step - loss: 2.1343 - acc: 0.1888 - val_loss: 2.8464 - val_acc: 0.2381
Epoch 2/50
390/390 [==============================] - 211s 540ms/step - loss: 1.7923 - acc: 0.3154 - val_loss: 5.3165 - val_acc: 0.1334
Epoch 3/50
390/390 [==============================] - 211s 540ms/step - loss: 1.6473 - acc: 0.3764 - val_loss: 1.8597 - val_acc: 0.3618
Epoch 4/50
390/390 [==============================] - 211s 540ms/step - loss: 1.5325 - acc: 0.4300 - val_loss: 2.0047 - val_acc: 0.3915
Epoch 5/50
390/390 [==============================] - 211s 540ms/step - loss: 1.4241 - acc: 0.4759 - val_loss: 1.6040 - val_acc: 0.4594
Epoch 6/50
390/390 [==============================] - 211s 540ms/step - loss: 1.3234 - acc: 0.5178 - val_loss: 2.0241 - val_acc: 0.3772
Epoch 7/50
390/390 [==============================] - 211s 540ms/step - loss: 1.2434 - acc: 0.5476 - val_loss: 1.3685 - val_acc: 0.5595
Epoch 8/50
390/390 [============================

In [2]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

NameError: ignored

In [17]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('DNST_model.h5')